# AI Research Pipeline Flow

## Overview
This notebook implements an automated research pipeline that performs web searches, generates a report, and emails the results.

## Components and Flow

### 1. Setup and Initialization
- **Library Imports**
  - agents (Agent, WebSearchTool, trace, Runner)
  - sendgrid for email functionality
  - dotenv for environment variables
  - asyncio for asynchronous operations
  
- **Agent Types Defined**
  - `search_agent`: Web search operations
  - `planner_agent`: Search strategy planning
  - `email_agent`: Email handling and delivery
  - `writer_agent`: Report generation

### 2. Agent Configurations

#### Search Agent
- Uses WebSearchTool with low context size
- Generates concise 300-word summaries
- Model: gpt-4o-mini
- Required tool choice setting

#### Planner Agent
- Plans 3 strategic searches
- Uses structured outputs via Pydantic models:
  - `WebSearchItem`
  - `WebSearchPlan`
- Outputs organized search plans

#### Email Agent
- SendGrid integration
- HTML email formatting
- Automated delivery system
- Custom subject line handling

#### Writer Agent
- 1000+ word detailed reports
- Structured via ReportData model
- Includes:
  - Short summary
  - Markdown report
  - Follow-up questions

### 3. Execution Flow
```python
with trace("Research trace"):
    1. Plan searches (plan_searches)
    2. Execute searches (perform_searches)
    3. Write report (write_report)
    4. Send email (send_email)
```

### 4. Core Functions

#### Research Functions
- `plan_searches(query: str)`: Search strategy generation
- `perform_searches(search_plan: WebSearchPlan)`: Parallel search execution
- `search(item: WebSearchItem)`: Individual search operations
- `write_report(query: str, search_results: list[str])`: Report synthesis
- `send_email(report: ReportData)`: Email delivery

## Technical Implementation
- Asynchronous execution using `async/await`
- Structured error handling
- Tracing for monitoring and debugging
- Parallel search operations via `asyncio.gather`
- Pydantic models for type safety

In [14]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [15]:
load_dotenv(override=True)  # Load environment variables from .env file

True

In [16]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [17]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of November 2025, several advanced AI agent frameworks have emerged, enhancing the development and deployment of autonomous systems:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and supporting complex interactions like multi-agent scenarios and dynamic workflows. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Polymorphic Combinatorial Framework (PCF)**: Utilizes LLMs and mathematical frameworks to design adaptive AI agents for complex, dynamic environments. PCF enables real-time parameter reconfiguration through combinatorial spaces, supporting scalable, dynamic, explainable, and ethical AI applications. ([arxiv.org](https://arxiv.org/abs/2508.01581?utm_source=openai))

- **GoalfyMax**: A protocol-driven multi-agent system introducing a standardized Agent-to-Agent communication layer based on the Model Context Protocol (MCP). It incorporates an Experience Pack architecture for structured knowledge retention and continual learning, enhancing adaptability and coordination in complex tasks. ([arxiv.org](https://arxiv.org/abs/2507.09497?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data for Agent Foundation Models and introduces novel strategies for agent test-time reflection and voting to enhance robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **Amazon Bedrock AgentCore**: A platform unveiled by AWS to simplify the development and deployment of advanced AI agents. It includes modular services supporting the full production lifecycle, such as scalable serverless deployment, context management, secure service access, tool integration, and enhanced problem-solving capabilities. ([techradar.com](https://www.techradar.com/pro/aws-looks-to-super-charge-ai-agents-with-amazon-bedrock-agentcore?utm_source=openai))

- **Kruti**: A multilingual AI agent developed by Ola Krutrim, designed to perform real-world tasks like booking taxis and ordering food by integrating directly with various online services. It is notable for understanding and responding in multiple Indian languages. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

- **Manus**: An autonomous AI agent developed by Butterfly Effect Technology, capable of independent reasoning, dynamic planning, and autonomous decision-making. Launched in March 2025, Manus is one of the first fully autonomous AI agents designed to execute complex real-world tasks without continuous human supervision. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Manus_%28AI_agent%29?utm_source=openai))

These frameworks represent significant advancements in AI agent development, offering enhanced capabilities for complex, dynamic, and autonomous task execution. 

In [18]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [19]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find up-to-date information on the newest AI agent frameworks being developed or released in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To uncover upcoming trends, features, and innovations in AI frameworks for agents in 2025.', query='AI agent frameworks trends 2025'), WebSearchItem(reason='To identify specific examples of AI agent frameworks launched or popularized in 2025.', query='new AI agent frameworks released 2025')]


In [20]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("swapnilpuri@gmail.com") 
    to_email = To("swapnilpuri@gmail.com") 
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [21]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given subject and HTML body', params_json_schema={'properties': {'subject': {'title': 'Subject', 'type': 'string'}, 'html_body': {'title': 'Html Body', 'type': 'string'}}, 'required': ['subject', 'html_body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x00000138FB766E80>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [22]:
INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [23]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [24]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [25]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an email with the report """
    print("Writing email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [26]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Hooray!")

Starting research...
Planning searches...
Will perform 3 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Writing email...
Email sent
Hooray!
